In [ ]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('SparkApp').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [2]:
retaildata = spark.read.option("inferSchema","true").option("header","true").csv("online-retail-dataset.csv")
retaildata.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [3]:
from pyspark.sql.functions import sum 
retail_selection = retaildata.select("Country","StockCode", "Quantity")
retail_selection = retail_selection.groupBy("StockCode", "Country").agg(sum("Quantity").alias("Quantity_total"))
retail_selection.show()

+---------+--------------+--------------+
|StockCode|       Country|Quantity_total|
+---------+--------------+--------------+
|    22154|United Kingdom|          3646|
|    22478|United Kingdom|          1423|
|    22844|United Kingdom|           916|
|    21788|United Kingdom|           867|
|    22042|United Kingdom|           190|
|    22539|        France|           312|
|    85078|         Italy|            24|
|   85034C|         Italy|             4|
|    22389|United Kingdom|           202|
|    85087|United Kingdom|           304|
|   35809A|United Kingdom|           291|
|    84821|United Kingdom|           821|
|    22837|         Spain|            16|
|    21030|        Cyprus|            10|
|    84992|        Sweden|           240|
|    22713|   Switzerland|            12|
|    21989|       Belgium|            36|
|    21080|       Germany|           385|
|   84661C|United Kingdom|            71|
|   82613D|        Cyprus|            20|
+---------+--------------+--------

In [4]:
#slice Operation
from pyspark.sql.functions import col
retail_slice = retail_selection.where(col("Country") == "United Kingdom")
retail_slice.show()

#slice = datacube.where(col("Country") == "United States")

+---------+--------------+--------------+
|StockCode|       Country|Quantity_total|
+---------+--------------+--------------+
|    22154|United Kingdom|          3646|
|    22478|United Kingdom|          1423|
|    22844|United Kingdom|           916|
|    21788|United Kingdom|           867|
|    22042|United Kingdom|           190|
|    22389|United Kingdom|           202|
|    85087|United Kingdom|           304|
|   35809A|United Kingdom|           291|
|    84821|United Kingdom|           821|
|   84661C|United Kingdom|            71|
|    90157|United Kingdom|            10|
|   84984D|United Kingdom|            13|
|    23326|United Kingdom|          2435|
|    47016|United Kingdom|             2|
|    22768|United Kingdom|           807|
|    16238|United Kingdom|          1113|
|    22916|United Kingdom|          1667|
|    21704|United Kingdom|          4743|
|        M|United Kingdom|          3662|
|   35271S|United Kingdom|             6|
+---------+--------------+--------

In [5]:
#Dice Operation
retail_dice = retail_selection.where((col("Country") == "United Kingdom") | (col("Country") == "USA")).where((col("Quantity_total") == 0) | (col("Quantity_total") == "1"))
retail_dice.show()

#dice = datacube.where(((col("Country") == "United States") | (col("Country") == "United Kingdom")) & (col("Quantity") == 0))

+---------+--------------+--------------+
|StockCode|       Country|Quantity_total|
+---------+--------------+--------------+
|    21577|           USA|             0|
|   84706D|United Kingdom|             1|
|    21763|United Kingdom|             0|
|    21054|United Kingdom|             1|
|    23843|United Kingdom|             0|
|    22016|United Kingdom|             1|
|    22050|           USA|             0|
|   72802c|United Kingdom|             1|
|    23602|United Kingdom|             1|
|   90187A|United Kingdom|             1|
|   46776b|United Kingdom|             1|
|    21668|           USA|             0|
|    21669|           USA|             0|
|   90060B|United Kingdom|             1|
|   84031b|United Kingdom|             1|
|   35597B|United Kingdom|             1|
|    72815|United Kingdom|             0|
|    22649|           USA|             0|
|    21058|           USA|             0|
|   90126A|United Kingdom|             1|
+---------+--------------+--------

In [7]:
#Pivot Operation. We will make a new column for every country
retail_pivot = retaildata.groupBy("StockCode").pivot("Country").agg(sum("Quantity"))
retail_pivot.show()


#How many instances of each product were sold ineach country?
retail_pivot_2 = retaildata.groupBy("StockCode").pivot("Country").agg(sum("Quantity")).orderBy("StockCode")
retail_pivot_2.show()

#retail_data.groupBy("StockCode").pivot("Country").sum("Quantity").show()

+---------+---------+-------+-------+-------+------+------+---------------+------+--------------+-------+----+------------------+-------+------+-------+------+---------+-------+------+-----+-----+-------+---------+-----+-----------+------+------+--------+----+------------+---------+-----+------+-----------+----+--------------------+--------------+-----------+
|StockCode|Australia|Austria|Bahrain|Belgium|Brazil|Canada|Channel Islands|Cyprus|Czech Republic|Denmark|EIRE|European Community|Finland|France|Germany|Greece|Hong Kong|Iceland|Israel|Italy|Japan|Lebanon|Lithuania|Malta|Netherlands|Norway|Poland|Portugal| RSA|Saudi Arabia|Singapore|Spain|Sweden|Switzerland| USA|United Arab Emirates|United Kingdom|Unspecified|
+---------+---------+-------+-------+-------+------+------+---------------+------+--------------+-------+----+------------------+-------+------+-------+------+---------+-------+------+-----+-----+-------+---------+-----+-----------+------+------+--------+----+------------+---

+---------+---------+-------+-------+-------+------+------+---------------+------+--------------+-------+----+------------------+-------+------+-------+------+---------+-------+------+-----+-----+-------+---------+-----+-----------+------+------+--------+----+------------+---------+-----+------+-----------+----+--------------------+--------------+-----------+
|StockCode|Australia|Austria|Bahrain|Belgium|Brazil|Canada|Channel Islands|Cyprus|Czech Republic|Denmark|EIRE|European Community|Finland|France|Germany|Greece|Hong Kong|Iceland|Israel|Italy|Japan|Lebanon|Lithuania|Malta|Netherlands|Norway|Poland|Portugal| RSA|Saudi Arabia|Singapore|Spain|Sweden|Switzerland| USA|United Arab Emirates|United Kingdom|Unspecified|
+---------+---------+-------+-------+-------+------+------+---------------+------+--------------+-------+----+------------------+-------+------+-------+------+---------+-------+------+-----+-----+-------+---------+-----+-----------+------+------+--------+----+------------+---

In [8]:
#Rollup. Divide the quantity to +ve and -ve
retail_rollUp = retail_selection.select("StockCode","Country",(col("Quantity_total") > 0).alias("positive_quantity"))
retail_rollUp.show()
#roll_up = datacube.rollup("Stock Code", "Country, (col("Quantity") > 0).alias("positive quantity")

+---------+--------------+-----------------+
|StockCode|       Country|positive_quantity|
+---------+--------------+-----------------+
|    22154|United Kingdom|             true|
|    22478|United Kingdom|             true|
|    22844|United Kingdom|             true|
|    21788|United Kingdom|             true|
|    22042|United Kingdom|             true|
|    22539|        France|             true|
|    85078|         Italy|             true|
|   85034C|         Italy|             true|
|    22389|United Kingdom|             true|
|    85087|United Kingdom|             true|
|   35809A|United Kingdom|             true|
|    84821|United Kingdom|             true|
|    22837|         Spain|             true|
|    21030|        Cyprus|             true|
|    84992|        Sweden|             true|
|    22713|   Switzerland|             true|
|    21989|       Belgium|             true|
|    21080|       Germany|             true|
|   84661C|United Kingdom|             true|
|   82613D

In [ ]:
#Drill Down. D

In [9]:
busbreakdownData = spark.read.option("inferSchema","true").option("header","true").csv("busbreakdownData.csv")
busbreakdownData.show()

+-----------+---------------+--------+------+------------+---------------+----------------+------+
|School year|Busbreakdown_ID|Run_Type|Bus_No|Route_Number|School_Serviced|     Occurred_On|  Town|
+-----------+---------------+--------+------+------------+---------------+----------------+------+
|       2010|          45345|      43|     1|           4|            dfg|12-01-2010 08:26|Bhopal|
|       2012|           5756|       4|     2|           5|           dgdf|12-02-2010 08:26|Mumbai|
|       2020|            223|      43|     3|           6|           dgdf|12-01-2011 08:26|  Bang|
+-----------+---------------+--------+------+------------+---------------+----------------+------+



In [18]:
datacube = busbreakdownData.select("Busbreakdown_ID","Town", "Route_Number","School year")
from pyspark.sql.functions import sum,count

In [19]:
#How many breakdowns occurred in total?
datacube.agg(count("Busbreakdown_ID")).show()

+----------------------+
|count(Busbreakdown_ID)|
+----------------------+
|                     3|
+----------------------+



In [26]:
#How many breakdowns occurred in each town in total?
datacube.groupBy("Town").agg(count("Busbreakdown_ID")).show()

+------+----------------------+
|  Town|count(Busbreakdown_ID)|
+------+----------------------+
|Mumbai|                     1|
|  Bang|                     1|
|Bhopal|                     1|
+------+----------------------+



In [27]:
#How many breakdowns occurred in each route number in total?
datacube.groupBy("Town").agg(count("Busbreakdown_ID")).show()


+------+----------------------+
|  Town|count(Busbreakdown_ID)|
+------+----------------------+
|Mumbai|                     1|
|  Bang|                     1|
|Bhopal|                     1|
+------+----------------------+



In [28]:
How many breakdowns occurred in each town per year?

retaildata_cube = retaildata.cube("Town","School year").groupBy("Town").agg(count("Busbreakdown_ID")).show()

Object `year` not found.


AttributeError: 'GroupedData' object has no attribute 'groupBy'

In [ ]:
#How many breakdowns occurred in each route number per year?
retaildata.cube("name", df.age).count().orderBy("name", "age").show()

In [37]:
datacube.cube("Town", "School year", "Route_Number")\
            .count()\
            .withColumnRenamed("count(Busbreakdown_ID)", "Count")\
            .orderBy("School year", "Town","Route_Number")\
            .show()

+------+-----------+------------+-----+
|  Town|School year|Route_Number|count|
+------+-----------+------------+-----+
|  null|       null|        null|    3|
|  null|       null|           4|    1|
|  null|       null|           5|    1|
|  null|       null|           6|    1|
|  Bang|       null|        null|    1|
|  Bang|       null|           6|    1|
|Bhopal|       null|        null|    1|
|Bhopal|       null|           4|    1|
|Mumbai|       null|        null|    1|
|Mumbai|       null|           5|    1|
|  null|       2010|        null|    1|
|  null|       2010|           4|    1|
|Bhopal|       2010|        null|    1|
|Bhopal|       2010|           4|    1|
|  null|       2012|        null|    1|
|  null|       2012|           5|    1|
|Mumbai|       2012|        null|    1|
|Mumbai|       2012|           5|    1|
|  null|       2020|        null|    1|
|  null|       2020|           6|    1|
+------+-----------+------------+-----+
only showing top 20 rows



In [39]:
datacube.cube("Town", "School year", "Route_Number")\
            .count()\
            .orderBy("School year", "Town","Route_Number")\
            .show()

+------+-----------+------------+-----+
|  Town|School year|Route_Number|count|
+------+-----------+------------+-----+
|  null|       null|        null|    3|
|  null|       null|           4|    1|
|  null|       null|           5|    1|
|  null|       null|           6|    1|
|  Bang|       null|        null|    1|
|  Bang|       null|           6|    1|
|Bhopal|       null|        null|    1|
|Bhopal|       null|           4|    1|
|Mumbai|       null|        null|    1|
|Mumbai|       null|           5|    1|
|  null|       2010|        null|    1|
|  null|       2010|           4|    1|
|Bhopal|       2010|        null|    1|
|Bhopal|       2010|           4|    1|
|  null|       2012|        null|    1|
|  null|       2012|           5|    1|
|Mumbai|       2012|        null|    1|
|Mumbai|       2012|           5|    1|
|  null|       2020|        null|    1|
|  null|       2020|           6|    1|
+------+-----------+------------+-----+
only showing top 20 rows

